In [ ]:
from pathlib import Path
import pandas as pd


In [ ]:
filename = Path('./data/processed/20210818_evening_toilet_visits.csv')
df = pd.read_csv(filename)

df


In [ ]:
vist_times = df['visit_time'][df['visit_time'].notnull()]
ax = vist_times.plot.hist(bins=10, alpha=0.5)



In [ ]:
%%capture --no-stdout
from scipy import stats

list_of_dists = [nm for nm, _ in stats._distr_params.distcont]

results = []

for i in list_of_dists:
    dist = getattr(stats, i)
    try:
        param = dist.fit(vist_times)
        a = stats.kstest(vist_times, i, args=param)
    except:
        print(f'Could not fit {i}')
        continue
    results.append((i,a[0],a[1], param))
        
results.sort(key=lambda x:float(x[2]), reverse=True)
    

In [ ]:
for j in results:
    print("{}: statistic={}, pvalue={}".format(j[0], j[1], j[2]))
    
with open(Path('./data/stats/toilet_fits.txt'), 'w') as f:
    f.write('name, statistic, pvalue, params\n')
    for j in results:
        f.write(f'{j[0]}, {j[1]:.4f}, {j[2]:.4f}, {j[3]}\n')

## Activity times

In [ ]:
filename = Path('./data/processed/20210818_evening_staff_activities.csv')
df = pd.read_csv(filename, parse_dates=['start_time', 'end_time'])

df

In [ ]:
import numpy as np

activitiy_times = np.array((df['end_time'] - df['start_time'])/np.timedelta64(1, 's'))

In [ ]:
import matplotlib.pyplot as plt
hist = plt.hist(activitiy_times, bins=15)
print(np.mean(activitiy_times))

In [ ]:
%%capture --no-stdout
from scipy import stats

list_of_dists = [nm for nm, _ in stats._distr_params.distcont]

results = []

for i in list_of_dists:
    dist = getattr(stats, i)
    try:
        param = dist.fit(activitiy_times)
        a = stats.kstest(activitiy_times, i, args=param)
    except:
        print(f'Could not fit {i}')
        continue
    results.append((i,a[0],a[1], param))
        
results.sort(key=lambda x:float(x[2]), reverse=True)

In [ ]:
for j in results:
    print("{}: statistic={}, pvalue={}".format(j[0], j[1], j[2]))
    
with open(Path('./data/stats/activitiy_time_fits.txt'), 'w') as f:
    f.write('name, statistic, pvalue, params\n')
    for j in results:
        f.write(f'{j[0]}, {j[1]:.4f}, {j[2]:.4f}, {j[3]}\n')